In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import sys
import os 
from torchvision import datasets,transforms,models,utils
from torch.utils.data import DataLoader,ConcatDataset,Dataset
import numpy as np
import matplotlib.pyplot as plt
import math
import re 
import timm
import pandas as pd
import numpy as np
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Dataset

In [ ]:
transform_original = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_flipped = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.RandomHorizontalFlip(p=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

spoof_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
    transforms.RandomHorizontalFlip(p=1),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
from PIL import Image
class CustomDataset(Dataset):
    def __init__(self,root,special_transform=None,general_transform=None,special_classes=None):
        self.root=root
        self.labels=self.assign_labels()
        if special_classes is not None:
            if not isinstance(special_classes,(set,list,tuple)):
                raise TypeError("special_classes must be a set,list or tuple")
            self.special_classes=set(special_classes)
        else:
            self.special_classes=set()
        self.special_transform=special_transform
        self.general_transform=general_transform
        
    def assign_labels(self):
        labels=[]
        for parent_folder in os.listdir(self.root):
            parent_folder_path=os.path.join(self.root,parent_folder)
            if os.path.isdir(parent_folder_path):
                label=1 if parent_folder=="real" else 0
                image_files=os.listdir(parent_folder_path)
                for image_file in image_files:
                    image_path=os.path.join(parent_folder_path,image_file)
                    labels.append((image_path,label))
        return labels
                
            
    def __getitem__(self,index):
        image_path,label=self.labels[index]
        image = Image.open(image_path).convert("RGB")
        if label in self.special_classes and self.special_transform:
            image=self.special_transform(image)
        elif self.general_transform:
            image=self.general_transform(image)
        return image,label
    def __len__(self):
        return len(self.labels)

In [ ]:
EPOCHS=3
BATCH_SIZE=16
EPOCH_LEN=len(str(EPOCHS))

In [ ]:
train_orig=CustomDataset("/kaggle/input/combineeee/Combine/train",general_transform=transform_original)
train_flip = CustomDataset(root="/kaggle/input/combineeee/Combine/train",
                           general_transform=transform_flipped,
                           special_transform=spoof_transforms,
                           special_classes=[0])
train_data_combined = ConcatDataset([train_orig, train_flip])
train_loader = DataLoader(train_data_combined, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
val_orig = CustomDataset("/kaggle/input/combineeee/Combine/val", general_transform=transform_original)
val_flip = CustomDataset(root="/kaggle/input/combineeee/Combine/val",
                           general_transform=transform_flipped,
                           special_transform=spoof_transforms,
                           special_classes=[0])

val_data_combined = ConcatDataset([val_orig, val_flip])
val_loader = DataLoader(val_data_combined, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
test_orig = CustomDataset("/kaggle/input/combineeee/Combine/test", general_transform=transform_original)
test_loader = DataLoader(test_orig, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
print(len(train_data_combined),len(val_data_combined),len(test_orig))

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Training

In [ ]:
def train_one_epoch(model, train_loader, device, optimizer, criterion, scheduler_lr=None):
    model.train()
    total_loss, total_f1, total_correct, total_samples, total_grad = 0, 0, 0, 0, 0

    for idx, (train_x_data, train_y_data) in enumerate(train_loader):
        train_x_data, train_y_data = train_x_data.to(device), train_y_data.to(device)
        optimizer.zero_grad()
        y_pred = model(train_x_data)

        loss = criterion(y_pred, train_y_data)
        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
        optimizer.step()

        _, y_pred_max = torch.max(y_pred, dim=1)
        f1_score_val = f1_score(train_y_data.cpu(), y_pred_max.cpu(), average="macro")
        total_correct += (y_pred_max == train_y_data).sum().item()
        total_samples += train_y_data.size(0)
        total_loss += loss.item()
        total_f1 += f1_score_val.item()
        total_grad += grad_norm

        if idx % 100 == 0 and idx > 0:
            lr = optimizer.param_groups[0]['lr']
            print(
                f'[{e:>{EPOCH_LEN}}/{EPOCHS:>{EPOCH_LEN}}] [{idx}/{len(train_loader)}] '
                f'lr {lr:.10f} | loss {loss:.10f} ({total_loss/idx:.4f}) | '
                f'grad_norm {grad_norm:.4f} ({total_grad/idx:.4f}) | '
            )

    if scheduler_lr:
        scheduler_lr.step()
    avg_loss = total_loss / len(train_loader)
    avg_f1 = total_f1 / len(train_loader)
    avg_grad = total_grad / len(train_loader)
    accuracy = total_correct / total_samples
    return avg_loss, avg_f1, accuracy, avg_grad

In [ ]:
def validate_one_epoch(model, val_loader, device, criterion, scheduler_lr=None):
    model.eval()
    total_loss, total_f1, total_correct, total_samples = 0, 0, 0, 0

    with torch.no_grad():
        for val_x_data, val_y_data in val_loader:
            val_x_data, val_y_data = val_x_data.to(device), val_y_data.to(device)
            y_val_pred = model(val_x_data)
            val_loss = criterion(y_val_pred, val_y_data)

            _, y_val_pred_max = torch.max(y_val_pred, dim=1)
            f1_score_val = f1_score(val_y_data.cpu(), y_val_pred_max.cpu(), average="macro")
            total_correct += (y_val_pred_max == val_y_data).sum().item()
            total_samples += val_y_data.size(0)
            total_loss += val_loss.item()
            total_f1 += f1_score_val.item()

    avg_loss = total_loss / len(val_loader)
    if scheduler_lr:
        scheduler_lr.step(avg_loss)
    avg_f1 = total_f1 / len(val_loader)
    accuracy = total_correct / total_samples
    return avg_loss, avg_f1, accuracy

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import timm
from timm.models.layers import trunc_normal_
model = timm.create_model('vit_base_patch16_224.augreg_in21k_ft_in1k',pretrained=True)
model.head = torch.nn.Linear(model.head.in_features, 2)
trunc_normal_(model.head.weight, mean=0.0, std=0.02)
model = model.to(device)

In [ ]:
from sklearn.metrics import f1_score
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import time
EPOCHS=3
criterion=nn.CrossEntropyLoss()

lr = 3e-5
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler_linear = lr_scheduler.LinearLR(optimizer, start_factor=0.01, total_iters=10)
scheduler_cosine = lr_scheduler.CosineAnnealingLR(optimizer, T_max=490, eta_min=lr/100)
scheduler_lr = lr_scheduler.SequentialLR(optimizer, [scheduler_linear,scheduler_cosine],milestones=[10])

start_time = time.time()
for e in range(1, EPOCHS + 1):
    avg_loss_train, avg_f1_train, avg_accuracy_train, avg_grad = train_one_epoch(model, train_loader, device,
                                                                           optimizer, criterion, scheduler_lr)
    avg_loss_val, avg_f1_val, avg_accuracy_val = validate_one_epoch(model, val_loader, device, criterion)

    time_taken = time.time() - start_time
    time_format = time.strftime("%H:%M:%S", time.gmtime(time_taken))

    print(
    f"[{e:>{EPOCH_LEN}}/{EPOCHS:>{EPOCH_LEN}}] Loss: {avg_loss_train:.5f} | "
    + f"F1-score: {avg_f1_train:.3f} | Acc: {avg_accuracy_train:.3f} | "
    + f"Val Loss: {avg_loss_val:.3f} | Val F1: {avg_f1_val:.3f} | "
    + f"Val Acc: {avg_accuracy_val:.3f} | {time_format}s | "
    + f"Grad: {avg_grad:.5f}"
    )


In [ ]:
torch.save(model,'modelVITcombine.pth')

In [ ]:
from IPython.display import FileLink
FileLink(r"modelVITcombine.pth")

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)

        tp += (pred.eq(1) & target.eq(1).view_as(pred)).sum().item()
        tn += (pred.eq(0) & target.eq(0).view_as(pred)).sum().item()
        fp += (pred.eq(1) & target.eq(0).view_as(pred)).sum().item()
        fn += (pred.eq(0) & target.eq(1).view_as(pred)).sum().item()

        correct += pred.eq(target.view_as(pred)).sum().item()

    accuracy = correct / len(test_loader.dataset)
    far = fp / (fp + tn)
    frr = fn / (fn + tp)
    recall = tp / (tp + fn)

    hter = (far + frr ) / 2

    print(f"test acc: {accuracy * 100}%")
    print(f"recall: {recall * 100}%")
    print(f"far: {far * 100}%")
    print(f"frr: {frr * 100}%")
    print(f"hter: {hter * 100}%")